# High-throughput data matrix reading for COVID-19 testing in Singapore

Minimum viable product would be a Python package that you can install on your computer. If successful, there's potential to deploy as a web app on a server. I think there's potential for use on mobile platforms, take picture and upload.

To host this as a web app, a few options are available:
- MEAN stack
- Flask/Bottle
- Onsen

There's a bug regarding [bounding box generation](https://github.com/NaturalHistoryMuseum/pylibdmtx/issues/51). Makes me wonder if it's worth fixing the bug, or segmenting the image, and processing them separately. I decided on the latter.

## Dependencies

See below, along with the regular NumPy, matplotlib libraries. I recommend installing the Python flavour of OpenCV rather than using the bare distributions. You will need to install the libdmtx package using your system package manager.

In [ ]:
import cv2
from pylibdmtx.pylibdmtx import decode
from PIL import Image
import math
import os

## Method 1: Contour Detection
### Image preprocessing

We'll use contours and bounding rectangles to segment out areas of interest after some basic image preprocessing.

First, we’ll convert the image to grayscale and will perform a Gaussian blur on it to simplify it and to remove noise.

Then, we'll perform a binary threshold on the preprocessed image. Since the data matrices are pasted on white circles on the sample tubes, I postulate the threshold can pick up the lighter circles as features of interest in the image.

**Future work.** Might have to look into adaptive thresholding to deal with images with variable lighting.

In [ ]:
plt.figure(dpi=400).patch.set_facecolor('white')

samples = cv2.imread('images/3.jpeg')
samples_grey = cv2.cvtColor(samples, cv2.COLOR_BGR2GRAY)
plt.subplot(131); plt.title('greyscale'); plt.axis('off'); plt.xticks([]); plt.yticks([])
plt.imshow(samples_grey, cmap = plt.cm.gray)

samples_gaussian = cv2.GaussianBlur(samples_grey, (5, 5), 0)
plt.subplot(132); plt.title('gaussian'); plt.xticks([]); plt.yticks([])
plt.imshow(samples_gaussian, cmap = plt.cm.gray)

_, samples_binary = cv2.threshold(samples_gaussian, 170, 255, cv2.THRESH_BINARY)
plt.subplot(133); plt.title('binary'); plt.xticks([]); plt.yticks([])
plt.imshow(samples_binary, cmap = plt.cm.gray)

### Finding contours and constructing bounding rectangles

Now that simplified, binary versions of the image are available, we can use them to identiy feastures of interest. Using the provided functions in OpenCV, we'll find the contour of each coin. Using the `cv2.RETR_EXTERNAL` flag to the function returns only the external contours, which is the largest circle formed by the data matrix sticker. This prevents the algorithm from picking up on the smaller contours inside the data matrix pattern.

In addition, I also set a minimum and maximum size to limit the number of contours to a reasonable figure.

If it doesn't work, I'll play with the binary threshold parameters in the previous step to get better segmentation.

In [ ]:
plt.figure(dpi=400).patch.set_facecolor('white')

samples_contours, _ = cv2.findContours(samples_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

samples_and_contours = np.copy(samples)

min_area = 1000 # 1000
max_area = 1100 # 60000
large_contours = [cnt for cnt in samples_contours if cv2.contourArea(cnt) < max_area]
final_contours = [cnt for cnt in samples_contours if cv2.contourArea(cnt) > min_area]

cv2.drawContours(samples_and_contours, final_contours, -1, (255,0,0))
plt.subplot(131); plt.title('contours'); plt.axis('off'); plt.xticks([]); plt.yticks([])
plt.imshow(samples_and_contours)

bounding_img = np.copy(samples)

box_coordinates = []

c = 0
for contour in final_contours:
    x, y, w, h = cv2.boundingRect(contour)
    box_coordinates.append([x, y, w, h])
    cv2.rectangle(bounding_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

plt.subplot(132); plt.title('bounding_box'); plt.axis('off'); plt.xticks([]); plt.yticks([])
plt.imshow(bounding_img)
print('number of samples: %d' % len(final_contours))
    
col_sort = sorted(box_coordinates, key=lambda l:l[1])

# these are actually important constants, my implementation of auto-detection for grids is sketchy at best
row_length = 9
col_length = 9
col_taken = math.ceil(int(len(col_sort) / col_length)) + 1

labeled_img = np.copy(samples)

for i in range(col_taken):
    row = sorted(col_sort[i * col_length : i * col_length + row_length])
    for k, _ in enumerate(row):
        cv2.putText(labeled_img, f"({k}, {i})", (row[k][0], row[k][1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    
plt.subplot(133); plt.title('labeled'); plt.axis('off'); plt.xticks([]); plt.yticks([])
plt.imshow(labeled_img)

### Saving bounding rectangles as separate, labeled images

Now that we know this method works, at least for this image, we can work on saving the bounding rectangles as separate images, to be processed by libdmtx. It's nearly the same code as above, but instead of writing the grid coordinates into an image, we extract it, and save it as a file name.

In [ ]:
for i in range(col_taken):
    row = sorted(col_sort[i * col_length : i * col_length + row_length])
    for k, _ in enumerate(row):
        # data_matrix = samples[row[k][1]:row[k][1]+row[k][3], row[k][0]:row[k][0]+row[k][2]]
        data_matrix = samples[row[k][1]:row[k][1]+row[k][3] + 1, row[k][0]:row[k][0]+row[k][2] + 1]
        cv2.imwrite(f'output/{k}_{i}.jpg', data_matrix)
        
def display_multiple_img(images, rows = row_length, cols = col_length):
    os.chdir('output')
    figure, ax = plt.subplots(nrows=rows,ncols=cols,dpi=400)
    figure.patch.set_facecolor('white')
    for i, image in enumerate(images):
        ax.ravel()[i].imshow(cv2.imread(image))
        ax.ravel()[i].set_title(image)
        ax.ravel()[i].title.set_fontsize(4)
        ax.ravel()[i].set_axis_off()
    for i in range(len(images), (rows*cols)):
        ax.ravel()[i].set_axis_off()
    
    os.chdir('..')
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.05, hspace=1.0)
    plt.show()
    
    
images = [x for x in sorted(os.listdir('output'), key = lambda k: [k[2], k[0]]) if x.endswith('.jpg')]

display_multiple_img(images)

### Reading the data matrix codes

Here comes the simple part, hopefully everything works well with the library.


In [ ]:
def read_data_matrix(image_list):
    os.chdir('output')
    a = []
    for i in image_list:
        e = decode(cv2.imread(i),max_count=1)
        try:
            print(e[0][0], i)
        except IndexError:
            print('error', i)
        a.append(e)
    print(f'Images parsed: {len(a)}')
    os.chdir('..')
    # return info_dict

read_data_matrix(images)


## Method 2: Edge Detection with the Hough Circle Transform

Sometimes segmenting via color or intensity as we did with binary thresholding isn’t sufficient. What if we have to deal with variations of lighting intensity? There may be cases where the white circles representing the stickers aren't uniform throughout.

Edges are points in an image where there is a change in brightness or intensity, which usually implies a boundary between different objects. The [Canny edge detection algorithm](https://www.wikiwand.com/en/Canny_edge_detector) is a popular edge detection algorithm that produces accurate, clean edges.

After we process the image with the above algorithm, the [Hough Circle Transform](https://en.wikipedia.org/wiki/Circle_Hough_Transform) can be used to detect circles of variable size.

In [ ]:
os.chdir('..')

In [ ]:
def read_data_matrix(image_list):
    os.chdir('output')
    a = []
    for i in image_list:
        e = decode(cv2.imread(i)[:, :, :1],max_count=1)
        print(e, i)
        a.append(e)
    print(len(a))
    os.chdir('..')
    # return info_dict

read_data_matrix(images)

## Miscellaneous calculations

In [ ]:
y_coords = []
bounding_coordinates

for i in bounding_coordinates:
    y_coords.append(i[1])
    
inxi = sorted(y_coords)

m = 0
total = 0
for i in range(len(inxi) - 1):
    d = inxi[i + 1] - inxi[i]
    if d > m:
        m = d
    print(d)
    total += m

avg = total / len(inxi)

In [ ]:
col = 1
for i in range(len(inxi) - 1):
    d = inxi[i + 1] - inxi[i]
    if d > (avg / 2):
        col += 1

col

In [ ]:
os.chdir('..')

In [ ]:
i = decode(Image.open('images/3.jpeg'))

In [ ]:
len(i)

In [ ]:
i[1].rect.left

In [ ]:
i[0]

In [ ]:
m = decode(image)

In [ ]:
int(70/8) + 1

In [ ]:
# thresholding stuff

p = plt.imread('images/3.jpeg')
p = cv2.cvtColor(p, cv2.COLOR_RGB2GRAY)
#plt.subplot(151); plt.title('greyscale'); plt.imshow(p)

x, thr = cv2.threshold(p, 60, 255, cv2.THRESH_BINARY)
thr = thr.astype('uint8')
plt.figure(dpi=1200)
plt.plot(); plt.title('threshold')
plt.imshow(thr, cmap = plt.cm.gray)
plt.savefig('test.png')

In [ ]:
# finding bounding rectangles
image = Image.open('images/1.jpeg')
draw = ImageDraw.Draw(image)

for matrix in decode(image):
    rect = matrix.rect
    draw.rectangle(((rect.left, rect.top), (rect.left + rect.width, rect.top + rect.height)),outline='Aquamarine')
    
image.save('bounding_1.png')